# Conway's Game Of Life

*This notebook originally appeared as a post on*
[*Pythonic Perambulations*](http://jakevdp.github.io/blog/2013/08/07/conways-game-of-life/)
*by Jake Vanderplas.  The code and content is BSD-licensed.*

<!-- PELICAN_BEGIN_SUMMARY -->
In 1970 the British Mathematician John Conway created his "Game of Life" -- 
a set of rules that mimics the chaotic yet
patterned growth of a colony of biological organisms.  The "game" takes place on
a two-dimensional grid consisting of "living" and "dead" cells, and
the rules to step from generation to generation are simple:

- **Overpopulation:** if a living cell is surrounded by more than three living cells, it dies.
- **Stasis:** if a living cell is surrounded by two or three living cells, it survives.
- **Underpopulation:** if a living cell is surrounded by fewer than two living cells, it dies.
- **Reproduction:** if a dead cell is surrounded by exactly three cells, it becomes a live cell.

By enforcing these rules in sequential steps, beautiful and unexpected patterns can appear.

Because the Game of Life is so simple, the time step can be computed rather
tersely in Python.  Here I implement two possibilities: one using generator expressions,
and one using the ``convolve2d`` function from ``scipy``.  Note that neither of
these are extremely performant: they involve creating several temporary arrays,
and will not work well for large problems with many time steps.  Nevertheless,
the simplicity makes these functions very attractive, and they are absolutely sufficient
for the small examples we'll consider here:

Note that we've made a choice here about the game boundary.  Classically, the
game takes place on an infinite, flat plane.  Here, for simplicity, we've used
a torroidal geometry (likely familiar to players of 1980s computer games like
Asteroids), where the grid wraps from top to bottom and left to right.

In [ ]:
%pylab inline
%matplotlib inline

import time
import numpy as np

In [ ]:
# NOTA: para escribir por pantalla "normal" (con print() y caracteres)
# No usado por ahora en esta versión para Jupyter notebook...
def print_tablero(m, vacio="0", lleno="1", sep=""):
    """Escribe por pantalla la matriz m con formato de tablero de 0's y 1's"""
    cad = str(m).replace(",", "").replace(" ", "").replace("[", "").replace("]","\n")
    cad = cad.replace("0", vacio + sep).replace("1", lleno + sep)
    print(cad, end="")


def clear_screen():
    """Limpia la pantalla (en consola). Válido en Linux, Mac y Windows"""
    # NOTA: no sirve en modo interactivo, IPython, Jupyter notebook, etc.
    import os
    os.system('cls' if os.name=='nt' else 'clear')

In [ ]:
def num_vecinos(m, x, y):
    """Devuelve el número de vecinos de la celda m[y][x], como un valor de 0 a 8"""
    result = 0
    FILAS = len(m)
    COLUMNAS = len(m[0]) if len(m) else 0
    for i in [-1, 0, 1]:
        for j in [-1, 0, 1]:
            if i != 0 or j != 0:  # no contar la celda en sí!
                if ((x + i >= 0) and (x + i < COLUMNAS) and (y + j >= 0)
                         and (y + j < FILAS)):
                    result += m[y+j][x+i]

    return result


def sig_generacion(m):
    """Devuelve la matriz resultante de aplicar las reglas del juego a cada celda"""
    FILAS = len(m)
    COLUMNAS = len(m[0]) if len(m) else 0
    new_m = []  # matriz resultado
    for i in range(FILAS):
        l = []  # Una lista para ir generando una fila
        for j in range(COLUMNAS):
            vec = num_vecinos(m, j, i)
            if vec < 2 or vec > 3:
                l.append(0)  # muere
            elif vec == 3:
                l.append(1)  # nace
            else:
                l.append(m[i][j])  # sobrevive si estaba viva
        new_m.append(l)

    return new_m

In [ ]:
# NOTA: no usada si se lee desde fichero, en esta versión (en otras se usa también en ese caso)
def leer_tablero(tablero_cadena):
    """Convierte una cadena adecuada en una matriz, para operar mejor con ella"""
    i = 0
    l = []  # Una lista para ir generando una fila
    m = []  # La matriz resultado
    columnas = 0
    for celda in tablero_cadena:
        if celda == '\n':
            if columnas == 0:
                columnas = len(l)
            elif columnas != len(l):
                print(f"Error en la fila {i}, faltan o sobran datos!")
                exit(1)
            i = i + 1
            m.append(l)
            l = []
        elif celda != " ":  # ignoramos los espacios (mejora por claridad)
            # Aceptamos varias cosas como "vacío"; cualquier otra implica "lleno"
            if celda in ".0-·":
                l.append(0)
            else:
                l.append(1)

    return m

In [ ]:
def leer_tablero_fich(nombre_fich):
    """Lee un fichero de texto con un contenido adecuado (...) y lo convierte
    en una matriz que representa el tablero de juego (0's y 1's numéricos)"""
    # Recordar que se debe cumplir que:
    # - Comprobar que todas tengan la misma logitud
    # - Permitir cualquier número de líneas de entrada
    # - Ignorar las líneas completamente vacías
    m = []  # La matriz resultado
    try:
        with open(nombre_fich) as f:
            for line in f:
                l = []
                line = line.strip()  # quitar espacios ini/fin y \n final
                line = line.replace(" ", "")  # Opc.: quitar espacios intermedios
                if line != "":  # ignorar las vacías
                    for carac in line:
                        if carac in ".0-·":  # Opc.: permitir "0"s y más...
                            l.append(0)
                        else:
                            l.append(1)
                    m.append(l)
    except:
        m = []  # en caso de error, mejor esto (por si estaba "a medias")
    return m

In [ ]:
figsize = 0  # variable global para controlar/guardar el estado de la imagen
im = None
fig = None
ax = None

def life_draw(X):
    """Produce a Game of Life Drawing
    X : array_like
        a two-dimensional numpy array showing the game board
    dpi : integer
        the number of dots per inch in the resulting animation.
        This controls the size of the game board on the screen
    """
    global dpi, im, figsize, fig, ax
    X = np.asarray(X)
    assert X.ndim == 2
    X = X.astype(bool)
    
    tmp_figsize = (X.shape[1] * 10. / dpi, X.shape[0] * 10. / dpi)

    if True: # figsize != tmp_figsize:
        figsize = tmp_figsize
        fig = plt.figure(figsize=figsize, dpi=dpi)
        ax = fig.add_axes([0, 0, 1, 1], xticks=[], yticks=[], frameon=False)
        im = ax.imshow(X, cmap=plt.cm.binary, interpolation='nearest')
        im.set_clim(-0.05, 1)  # Make background gray
    else:
        im.set_data(X)
    
    fig.canvas.draw()

In [ ]:
%pwd

In [ ]:
# Comienzo del programa

# NOTA: dependiendo de cómo/dónde lo ejecutemos, puede hacer falta ajustar la ruta!
FICH_TABL = "juego-vida-spc.txt"

# m = leer_tablero(tablero_texto)
m = leer_tablero_fich(FICH_TABL)
FILAS = len(m)
COLUMNAS = len(m[0]) if len(m) else 0

if FILAS == 0:
    print("ERROR LEYENDO FICHERO!")
else:
    # clear_screen()
    print(f"Tablero inicial ({FILAS} filas, {COLUMNAS} columnas):")
    # print_tablero(m, vacio="-", sep=" ")


# VARIABLES Y CONSTANTES para dibujar. dpi influye en el tamaño, ajustar si hace falta por la resol. de pantalla 
dpi = 5

## INITIALIZATION: START WITH THE MATRIX WE JUST CREATED
X = np.asarray(m)
assert X.ndim == 2
life_draw(X)

In [ ]:
## NEXT STEP: REPEAT TO SEE SUBSEQUENT GENERATIONS !!!
m = sig_generacion(m)
X = np.asarray(m)
life_draw(X)

m = sig_generacion(m)
X = np.asarray(m)
life_draw(X)

Let's give this a try with a random starting field:

In [ ]:
## INITIALIZATION: START WITH A RANDOM PATTERN (ONLY IN THE "CENTRAL" AREA)

# np.random.seed(0)
r = np.random.random((10, 40))
X = np.zeros((20, 60), dtype=bool)

X[5:15, 10:50] = (r > 0.75)
m = X.tolist()
life_draw(m)

In [ ]:
## NEXT STEP: REPEAT TO SEE SUBSEQUENT GENERATIONS !!!
m = sig_generacion(m)
X = np.asarray(m)
life_draw(X)

With the above random seed, the cells die off after about 40 generations.
In the process, some very interesting patterns show up: there are static
patterns, oscillating patterns, and a lot of spontaneous symmetry.  Let's
explore a few of the well-known patterns here:

# NOTA: la parte de aquí para abajo ahora no funciona (hacen falta más cambios)
## Static Configurations

Several static configurations are known: some of the smallest static units
are shown here.  We'll generate a few frames just to show that they are
in fact static.

In [ ]:
X = np.zeros((6, 21), dtype=bool)
X[2:4, 1:3] = 1
X[1:4, 5:9] = [[0, 1, 1, 0],
               [1, 0, 0, 1],
               [0, 1, 1, 0]]
X[1:5, 11:15] = [[0, 1, 1, 0],
                 [1, 0, 0, 1],
                 [0, 1, 0, 1],
                 [0, 0, 1, 0]]
X[1:4, 17:20] = [[1, 1, 0],
                 [1, 0, 1],
                 [0, 1, 0]]
m = X.tolist()
life_draw(X)

In [ ]:
## NEXT STEP (NO CHANGES IN THIS CASE)

m = sig_generacion(m)
life_draw(m)

## Some simple oscillators (The "Blinker" and the "Toad")

An oscillator is a pattern that returns to its initial configuration after some number
of steps.  The static patterns shown above could be thought of as oscillators with a
period of one.  Here are two commonly-seen period-two oscillators:

In [ ]:
blinker = [1, 1, 1]
toad = [[1, 1, 1, 0],
        [0, 1, 1, 1]]

X = np.zeros((6, 11), dtype=bool)
X[2, 1:4] = blinker
X[2:4, 6:10] = toad
m = X.tolist()
life_draw(X)

In [ ]:
## NEXT STEP (NO CHANGES IN THIS CASE)
m = sig_generacion(m)
life_draw(m)

## Another Oscillator: The "Pulsar"

More complicated oscillators exist.  Here's a period-three oscillator known as
"The Pulsar", which displays some appealing symmetry.

In [ ]:
X = np.zeros((17, 17), dtype=bool)
X[2, 4:7] = 1
X[4:7, 7] = 1
X += X.T
X += X[:, ::-1]
X += X[::-1, :]
m = X.tolist()
life_draw(X)

In [ ]:
## NEXT STEP
m = sig_generacion(m)
life_draw(m)

## The "Glider"

There are other classes of object which oscillate, but also move while oscillating.
One of the earliest seen is the "Glider", which after 4 steps returns to its
initial configuration, but shifted by one cell in both the x and y direction.
This is a configuration that often emerges from random starting points.

In [ ]:
glider = [[1, 0, 0],
          [0, 1, 1],
          [1, 1, 0]]
X = np.zeros((8, 8), dtype=bool)
X[:3, :3] = glider
m = X.tolist()
life_draw(X)

In [ ]:
## NEXT STEP
m = sig_generacion(m)
life_draw(m)

## Unbounded Growth

An early question posed about the Game of Life was whether any configurations exist which
result in asymptotically unbounded growth.  It was quickly found that the answer was yes.  Though
it wasn't the first discovered, the following is one of the most compact configurations which
display unbounded growth.  Note that this claim is precisely true only on an infinite game
board: using a torroidal (i.e. wrapping) geometry like we do here will lead to different
results, but the first several hundred generations are unaffected:

In [ ]:
unbounded = [[1, 1, 1, 0, 1],
             [1, 0, 0, 0, 0],
             [0, 0, 0, 1, 1],
             [0, 1, 1, 0, 1],
             [1, 0, 1, 0, 1]]
X = np.zeros((30, 40), dtype=bool)
X[15:20, 18:23] = unbounded
m = X.tolist()
life_draw(X)

In [ ]:
## NEXT STEP
m = sig_generacion(m)
life_draw(m)

## The "Gosper Glider Gun"

The earliest known instance of unbounded growth is one of my favorite configurations:
the "Glider Gun" discovered by Bill Gosper.  It is an oscillating pattern that creates
an infinite series of gliders.  It still amazes me that something like this can even
emerge from Conway's simple rules, but here it is.  We'll stop after a couple hundred
frames, but given an infinite game board this action would go on forever:

In [ ]:
glider_gun =\
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1],
 [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1],
 [1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
 [1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]

X = np.zeros((50, 70), dtype=bool)
X[1:10,1:37] = glider_gun
m = X.tolist()
life_draw(X)

In [ ]:
## NEXT STEP
m = sig_generacion(m)
life_draw(m)

## Going Further

Note that while the code above is well-suited for small explorations,
it is probably not sufficient to do very large and long game of life
computations.  For that, I'd recommend [Golly](http://golly.sourceforge.net/), an
open-source cross-platform package for computing and visualizing the Game of Life.
It has some nice optimizations, including a blazing fast hash-based computation of
generational steps for long-lived problems.

Diving further in, you might come across other very cool patterns.  One pattern, known as a "Breeder",
moves through space creating glider guns, which in turn create an endless series of
gliders.  Wikipedia has a [great animation](http://en.wikipedia.org/wiki/File:Conways_game_of_life_breeder_animation.gif)
of this in action:

<img src="http://upload.wikimedia.org/wikipedia/commons/e/e6/Conways_game_of_life_breeder_animation.gif">

Notice the series of glider guns, similar to the one we built above.
While this animation could certainly be created using the above Python code,
I'm just not sure I'd have the patience!


Despite (or perhaps because of) its simplicity, the Game of Life
has inspired an entire community of people who study its properties.  It has influenced fields
as diverse as mathematics, computer science, biology, epidemiology, and sociology.
This interest has led to the discovery of configurations with some very surprising properties.
Incredibly, it has even been shown that a Universal Turing Machine can be created within
the rules of the game of life.  That is, a computer which can compute game of life steps
could, in theory, use this process to compute just about anything!

Here are another few patterns you might try embedding in a game board, to see what will happen.

In [ ]:
diehard = [[0, 0, 0, 0, 0, 0, 1, 0],
           [1, 1, 0, 0, 0, 0, 0, 0],
           [0, 1, 0, 0, 0, 1, 1, 1]]

boat = [[1, 1, 0],
        [1, 0, 1],
        [0, 1, 0]]

r_pentomino = [[0, 1, 1],
               [1, 1, 0],
               [0, 1, 0]]

beacon = [[0, 0, 1, 1],
          [0, 0, 1, 1],
          [1, 1, 0, 0],
          [1, 1, 0, 0]]

acorn = [[0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0],
         [1, 1, 0, 0, 1, 1, 1]]

spaceship = [[0, 0, 1, 1, 0],
             [1, 1, 0, 1, 1],
             [1, 1, 1, 1, 0],
             [0, 1, 1, 0, 0]]

block_switch_engine = [[0, 0, 0, 0, 0, 0, 1, 0],
                       [0, 0, 0, 0, 1, 0, 1, 1],
                       [0, 0, 0, 0, 1, 0, 1, 0],
                       [0, 0, 0, 0, 1, 0, 0, 0],
                       [0, 0, 1, 0, 0, 0, 0, 0],
                       [1, 0, 1, 0, 0, 0, 0, 0]]

I hope you enjoyed this quick exploration! 
For more information on the wealth of information about this
game, you can browse the discussions and forums at
[Conway's Game of Life](http://conwaylife.com/)

*This post was written in an IPython notebook, which can be downloaded*
[*here*](http://jakevdp.github.io/downloads/notebooks/GameOfLife.ipynb),
*or viewed statically*
[*here*](http://nbviewer.ipython.org/url/jakevdp.github.io/downloads/notebooks/GameOfLife.ipynb).